In [ ]:
%%capture
%config Completer.use_jedi = False
%config InlineBackend.figure_formats = ['svg']
import os

STATIC_WEB_PAGE = {"EXECUTE_NB", "READTHEDOCS"}.intersection(os.environ)

# Install on Google Colab
import subprocess
import sys

from IPython import get_ipython

install_packages = "google.colab" in str(get_ipython())
if install_packages:
    for package in ["ampform[doc]", "graphviz"]:
        subprocess.check_call(
            [sys.executable, "-m", "pip", "install", package]
        )

In [ ]:
import logging
import warnings

logging.basicConfig()
logging.getLogger().setLevel(logging.ERROR)

warnings.filterwarnings("ignore")

# Analytic continuation

## Definitions

In [ ]:
import sympy as sp
from IPython.display import Math

{func}`.breakup_momentum_squared`:

In [ ]:
from ampform.dynamics import breakup_momentum_squared

s, m_a, m_b = sp.symbols("s, m_a, m_b")
q_squared = breakup_momentum_squared(s, m_a, m_b)

In [ ]:
Math(f"q^2(s) = {sp.latex(q_squared)}")

{func}`.phase_space_factor`:

In [ ]:
from ampform.dynamics import phase_space_factor

rho = phase_space_factor(s, m_a, m_b)

In [ ]:
rho_subs = rho.subs(4 * q_squared, 4 * sp.Symbol("q^{2}(s)"))
Math(fR"\hat{{\rho}}(s) = {sp.latex(rho_subs)}")

{func}`.phase_space_factor_ac` (analytic continuation):

In [ ]:
from ampform.dynamics import phase_space_factor_ac

rho_analytic = phase_space_factor_ac(s, m_a, m_b)

In [ ]:
from ampform.dynamics import _analytic_continuation

_analytic_continuation(
    sp.Symbol(R"\hat{\rho}"), s, s_threshold=(m_a + m_b) ** 2
)

## Interactive plot

In [ ]:
%matplotlib widget
import matplotlib.pyplot as plt
import mpl_interactions.ipyplot as iplt
import numpy as np

import symplot

In [ ]:
m = sp.Symbol("m", real=True)
rho = phase_space_factor(m ** 2, m_a, m_b)
rho_ac = phase_space_factor_ac(m ** 2, m_a, m_b)
np_rho, sliders = symplot.prepare_sliders(plot_symbol=m, expression=rho)
np_rho_ac = sp.lambdify((m, m_a, m_b), rho_ac, "numpy")

In [ ]:
plot_domain = np.linspace(0, 3, 1_000, dtype=np.complex64)
sliders.set_ranges(
    m_a=(0, 2, 200),
    m_b=(0, 2, 200),
)
sliders.set_values(
    m_a=0.45,
    m_b=1.4,
)

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(10, 4), tight_layout=True, sharey=True)
ax_abs, ax_real, ax_imag = axes
for ax in axes:
    ax.set_xlabel("$m$")
ylim = (0, 0.05)

ax_abs.set_title(R"$\left|\rho\right|$")
controls = iplt.plot(
    plot_domain,
    lambda *args, **kwargs: np.abs(np_rho(*args, **kwargs)),
    label="normal",
    **sliders,
    ylim=ylim,
    ax=ax_abs,
    linestyle="dotted",
)
iplt.plot(
    plot_domain,
    lambda *args, **kwargs: np.abs(np_rho_ac(*args, **kwargs)),
    label="analytic",
    controls=controls,
    ylim=ylim,
    ax=ax_abs,
    linestyle="dashed",
)
plt.legend(loc="upper right")

ax_real.set_title(R"Re($\rho$)")
iplt.plot(
    plot_domain,
    lambda *args, **kwargs: np_rho(*args, **kwargs).real,
    label="normal",
    controls=controls,
    ylim=ylim,
    ax=ax_real,
    linestyle="dotted",
)
iplt.plot(
    plot_domain,
    lambda *args, **kwargs: np_rho_ac(*args, **kwargs).real,
    label="analytic",
    controls=controls,
    ylim=ylim,
    ax=ax_real,
    linestyle="dashed",
)
plt.legend(loc="upper right")

ax_imag.set_title(R"Im($\rho$)")
iplt.plot(
    plot_domain,
    lambda *args, **kwargs: np_rho(*args, **kwargs).imag,
    label="normal",
    controls=controls,
    ylim=ylim,
    linestyle="dotted",
    ax=ax_imag,
)
iplt.plot(
    plot_domain,
    lambda *args, **kwargs: np_rho_ac(*args, **kwargs).imag,
    label="analytic",
    controls=controls,
    ylim=ylim,
    ax=ax_imag,
    linestyle="dashed",
)
plt.legend(loc="upper right")

plt.show()

{{ run_interactive }}

In [ ]:
if STATIC_WEB_PAGE:
    from IPython.display import SVG

    output_file = "analytic-continuation.svg"
    plt.savefig(output_file)
    display(SVG(output_file))